In [49]:
import json
from pathlib import Path
import gurobipy as gp
from gurobipy import GRB


In [50]:
# Function to load JSON data
def load_json(path: Path):
    with open(path, "r") as f:
        return json.load(f)

In [51]:
def run_flex_load_pv_optimization(bus_params, der_production, usage_preference, appliance_params):
    
    # Define model parameters
    T = len(der_production[0]["hourly_profile_ratio"])
    pv_max = der_production[0]["hourly_profile_ratio"]
    min_daily_energy = usage_preference[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
    
    PVcap = appliance_params["DER"][0]["max_power_kW"]
    max_import = bus_params[0]["max_import_kW"]
    max_export = bus_params[0]["max_export_kW"]
    import_tariff = bus_params[0]["import_tariff_DKK/kWh"]
    export_tariff = bus_params[0]["export_tariff_DKK/kWh"]
    energy_price = bus_params[0]["energy_price_DKK_per_kWh"]

    max_load = appliance_params["load"][0]["max_load_kWh_per_hour"]
    
    # Create optimization model
    m = gp.Model("flexible_load_pv")


    # Decision variables
    PD = {t: m.addVar(lb=0, ub=max_load, name=f"PD_{t}") for t in range(T)}
    PG = {t: m.addVar(lb=0, ub=pv_max[t]*PVcap, name=f"PG_{t}") for t in range(T)}
    PIMP = {t: m.addVar(lb=0, ub=max_import, name=f"PIMP_{t}") for t in range(T)}
    PEXP = {t: m.addVar(lb=0, ub=max_export, name=f"PEXP_{t}") for t in range(T)}
    PV_curtail = {t: m.addVar(lb=0, ub=PVcap, name=f"PV_curtail_{t}") for t in range(T)}



    # Constraints
    for t in range(T):
        m.addConstr(PIMP[t] - PEXP[t] == PD[t] - PG[t], name=f"balance_{t}")
        m.addConstr(PG[t] + PV_curtail[t] == pv_max[t] * PVcap, name=f"pv_cap_{t}")

    m.addConstr(gp.quicksum(PD[t] for t in range(T)) >= min_daily_energy, name="daily_min_energy")


    # Full objective function
    obj = gp.quicksum(
        energy_price[t] * (PIMP[t] - PEXP[t]) +
        import_tariff * PIMP[t] +
        export_tariff * PEXP[t] for t in range(T)
    )
    
    # Set objective 
    m.setObjective(obj, GRB.MINIMIZE)

    # Optimize model
    m.optimize()
    
    # Check if optimization is successful
    if m.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Solver ended with status {m.Status}")


    # Extract results
    results = []
    for t in range(T):
        imp = PIMP[t].X
        exp = PEXP[t].X
        pv_used = PG[t].X
        demand = PD[t].X
        net_grid = imp - exp
        pv_curt = PV_curtail[t].X
        results.append([t, imp, exp, pv_used, demand, net_grid, pv_curt])

    return results, m.ObjVal


In [52]:
def main():
    # File paths
    base_dir = Path().resolve().parent / "data" / "question_1a"

    # Load input data
    appliance_params = load_json(base_dir / "appliance_params.json")
    bus_params = load_json(base_dir / "bus_params.json")
    der_production = load_json(base_dir / "DER_production.json")
    usage_preference = load_json(base_dir / "usage_preference.json")
    
    # Run optimization
    results1a, obj_val1a = run_flex_load_pv_optimization(bus_params, der_production, usage_preference, appliance_params)


    #Change in values for a5
    

    # Change export tariff to 100 DKK/kWh, so that export is not an option
    bus_params_a5 = bus_params.copy()
    bus_params_a5[0]["export_tariff_DKK/kWh"] = 100
    
    resultsa5, obj_vala5 = run_flex_load_pv_optimization(bus_params_a5, der_production, usage_preference, appliance_params)
    
    # Change electricity prices to negative values

    
    
    # Print results
    #print(f"Objective (total cost): {obj_val:.2f} DKK\n")
    #print("Hour | Import | Export | PV_used | Demand | NetGrid | PV_Curtail")
    #print("-" * 70)
    #for row in results:
    #    print(f"{row[0]:>4} | {row[1]:>6.2f} | {row[2]:>6.2f} | {row[3]:>7.2f} | {row[4]:>6.2f} | {row[5]:>7.2f} | {row[6]:>9.2f}")
    #print("Total import:",sum(results[row][1] for row in range(len(results))))
    #print("Total consumption:", round(sum(results[row][4] for row in range(len(results)))))
    #
    ## Print for a5
    #print(f"Objective (total cost): {obj_vala5:.2f} DKK\n")
    #print("Hour | Import | Export | PV_used | Demand | NetGrid | PV_Curtail")
    #print("-" * 70)
    #for row in resultsa5:
    #    print(f"{row[0]:>4} | {row[1]:>6.2f} | {row[2]:>6.2f} | {row[3]:>7.2f} | {row[4]:>6.2f} | {row[5]:>7.2f} | {row[6]:>9.2f}")
    #print("Total import:",sum(resultsa5[row][1] for row in range(len(resultsa5))))
    #print("Total consumption:", round(sum(resultsa5[row][4] for row in range(len(resultsa5)))))
    
    # Print differences
    print("\nDifferences between original and a5 scenario:")
    print("Hour | Import Diff | Export Diff | PV_used Diff | Demand Diff | NetGrid Diff | PV_Curtail Diff")
    print("-" * 80)
    for row in range(len(results1a)):
        imp_diff = resultsa5[row][1] - results1a[row][1]
        exp_diff = resultsa5[row][2] - results1a[row][2]
        pv_used_diff = resultsa5[row][3] - results1a[row][3]
        demand_diff = resultsa5[row][4] - results1a[row][4]
        netgrid_diff = resultsa5[row][5] - results1a[row][5]
        pv_curtail_diff = resultsa5[row][6] - results1a[row][6]
        print(f"{row:>4} | {imp_diff:>11.2f} | {exp_diff:>11.2f} | {pv_used_diff:>13.2f} | {demand_diff:>11.2f} | {netgrid_diff:>12.2f} | {pv_curtail_diff:>15.2f}")
    
    
if __name__ == "__main__":
    main()
    
    
    
    

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 49 rows, 120 columns and 168 nonzeros
Model fingerprint: 0xe97ff5e5
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [2e-01, 1e+03]
  RHS range        [2e-01, 8e+00]
Presolve removed 33 rows and 66 columns
Presolve time: 0.01s
Presolved: 16 rows, 54 columns, 69 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.0800000e+03   1.498106e+04   0.000000e+00      0s
      13   -6.3725000e+00   0.000000e+00   0.000000e+00      0s

Solved in 13 iterations and 0.03 seconds (0.00 work units)
Optimal objective -6.372500000e+00
Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U 

In [53]:
def run_flex_load_with_discomfort(bus_params, der_production, usage_preference, appliance_params):
    T = len(der_production[0]["hourly_profile_ratio"])
    pv_max = der_production[0]["hourly_profile_ratio"]
    cap = appliance_params["DER"][0]["max_power_kW"]

    load_pref = usage_preference[0]["load_preferences"][0]

    # Reference profile handling
    if load_pref.get("hourly_profile_ratio") is not None:
        ref_profile = load_pref["hourly_profile_ratio"]
    else:
        min_daily_energy = load_pref["min_total_energy_per_day_hour_equivalent"]
        ref_profile = [min_daily_energy / T] * T
        print("⚠️ No reference profile in JSON, using flat profile instead.")

    ref_profile = [x * cap for x in ref_profile]

    #total_ref_energy = sum(ref_profile)

    
    max_import = bus_params[0]["max_import_kW"]
    max_export = bus_params[0]["max_export_kW"]
    import_tariff = bus_params[0]["import_tariff_DKK/kWh"]
    export_tariff = bus_params[0]["export_tariff_DKK/kWh"]
    energy_price = bus_params[0]["energy_price_DKK_per_kWh"]

    max_load = appliance_params["load"][0]["max_load_kWh_per_hour"]

    m = gp.Model("flexible_load_pv_1b")

    PD_flex = {t: m.addVar(lb=0, ub=max_load, name=f"PDflex_{t}") for t in range(T)}
    PG = {t: m.addVar(lb=0, ub=pv_max[t]*cap, name=f"PG_{t}") for t in range(T)}
    PIMP = {t: m.addVar(lb=0, ub=max_import, name=f"PIMP_{t}") for t in range(T)}
    PEXP = {t: m.addVar(lb=0, ub=max_export, name=f"PEXP_{t}") for t in range(T)}
    PV_curtail = {t: m.addVar(lb=0, ub=cap, name=f"PV_curtail_{t}") for t in range(T)}
    U = {t: m.addVar(lb=0, name=f"U_{t}") for t in range(T)}  # absolute deviation vars

    # Constraints
    for t in range(T):
        m.addConstr(PIMP[t] - PEXP[t] == PD_flex[t] - PG[t], name=f"balance_{t}")
        m.addConstr(PG[t] + PV_curtail[t] == pv_max[t] * cap, name=f"generation_{t}")
        #m.addConstr(PG[t] + PV_curtail[t] == pv_max[t], name=f"pv_cap_{t}")
        m.addConstr(U[t] >= PD_flex[t] - ref_profile[t], name=f"dev_pos_{t}")
        m.addConstr(U[t] >= -(PD_flex[t] - ref_profile[t]), name=f"dev_neg_{t}")

    # Conserve daily flexible energy equal to reference total
    #m.addConstr(gp.quicksum(PD_flex[t] for t in range(T)) == total_ref_energy, name="daily_energy_match")

    # Objective: cost + discomfort
    obj = gp.quicksum(
        energy_price[t] * (PIMP[t] - PEXP[t]) +
        import_tariff * PIMP[t] +
        export_tariff * PEXP[t] +
        U[t]  for t in range(T)
    )
    m.setObjective(obj, GRB.MINIMIZE)

    m.optimize()
    if m.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Solver ended with status {m.Status}")

    results = []
    for t in range(T):
        results.append([
            t,
            PIMP[t].X,
            PEXP[t].X,
            PG[t].X,
            PD_flex[t].X,
            U[t].X,
            PIMP[t].X - PEXP[t].X,
            PV_curtail[t].X
        ])

    return results, m.ObjVal


In [54]:
def main():
    # File paths
    base_dir = Path().resolve().parent / "data" / "question_1b"

    # Load input data
    appliance_params1b = load_json(base_dir / "appliance_params.json")
    bus_params1b = load_json(base_dir / "bus_params.json")
    der_production1b = load_json(base_dir / "DER_production.json")
    usage_preference1b = load_json(base_dir / "usage_preferences.json")
    
    # Run optimization
    results_battery1b, obj_val_battery1b = run_flex_load_with_discomfort(bus_params1b, der_production1b, usage_preference1b, appliance_params1b)

if __name__ == "__main__":
    main()

Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 96 rows, 144 columns and 240 nonzeros
Model fingerprint: 0x78f8009a
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [2e-01, 1e+03]
  RHS range        [1e-01, 3e+00]
Presolve removed 51 rows and 84 columns
Presolve time: 0.01s
Presolved: 45 rows, 60 columns, 105 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -7.0781850e+03   1.499919e+04   0.000000e+00      0s
      36    1.4251500e+01   0.000000e+00   0.000000e+00      0s

Solved in 36 iterations and 0.02 seconds (0.00 work units)
Optimal objective  1.425150000e+01


In [55]:
def run_flex_load_pv_optimization_w_battery(bus_params, der_production, usage_preference, appliance_params):
    
    # Define model parameters
    T = len(der_production[0]["hourly_profile_ratio"])
    pv_max = der_production[0]["hourly_profile_ratio"]
    #min_daily_energy = usage_preference[0]["load_preferences"][0]["min_total_energy_per_day_hour_equivalent"]
    
    load_pref = usage_preference[0]["load_preferences"][0]

    max_import = bus_params[0]["max_import_kW"]
    max_export = bus_params[0]["max_export_kW"]
    import_tariff = bus_params[0]["import_tariff_DKK/kWh"]
    export_tariff = bus_params[0]["export_tariff_DKK/kWh"]
    energy_price = bus_params[0]["energy_price_DKK_per_kWh"]
    
    max_load = appliance_params["load"][0]["max_load_kWh_per_hour"]
    PVcap = appliance_params["DER"][0]["max_power_kW"]
    
    charging_power = appliance_params["storage"][0]["max_charging_power_ratio"]
    discharging_power = appliance_params["storage"][0]["max_discharging_power_ratio"]
    efficiency_charge = appliance_params["storage"][0]["charging_efficiency"]
    efficiency_discharge = appliance_params["storage"][0]["discharging_efficiency"]
    battery_capacity = appliance_params["storage"][0]["storage_capacity_kWh"]
    
    #Calculations
    actual_charging = charging_power * battery_capacity
    actual_discharging = discharging_power * battery_capacity
    
    ref_profile = load_pref["hourly_profile_ratio"]
    ref_profile = [x * PVcap for x in ref_profile]
    
    
    # Create optimization model
    m = gp.Model("flexible_load_pv")


    # Decision variables
    PD_flex = {t: m.addVar(lb=0, ub=max_load, name=f"PDflex_{t}") for t in range(T)}
    PD = {t: m.addVar(lb=0, ub=max_load, name=f"PD_{t}") for t in range(T)}
    PG = {t: m.addVar(lb=0, ub=pv_max[t]*PVcap, name=f"PG_{t}") for t in range(T)}
    PIMP = {t: m.addVar(lb=0, ub=max_import, name=f"PIMP_{t}") for t in range(T)}
    PEXP = {t: m.addVar(lb=0, ub=max_export, name=f"PEXP_{t}") for t in range(T)}
    PV_curtail = {t: m.addVar(lb=0, ub=PVcap, name=f"PV_curtail_{t}") for t in range(T)}
    U = {t: m.addVar(lb=0, name=f"U_{t}") for t in range(T)} 
    
    #Battery decision variables
    P_charge = {t: m.addVar(lb=0, ub=actual_charging, name=f"charging_power_{t}") for t in range(T)}
    P_discharge = {t: m.addVar(lb=0, ub=actual_discharging, name=f"discharging_power_{t}") for t in range(T)}
    SOC = {t: m.addVar(lb=0, ub= battery_capacity, name=f"state_of_charge_{t}") for t in range(T)}

    # Battery start and stop
    m.addConstr(SOC[0]==battery_capacity * 0.5)
    m.addConstr(SOC[23] == SOC[0])


    # Constraints
    for t in range(T):
        m.addConstr(PIMP[t] - PEXP[t] == PD[t] - PG[t] + P_charge[t] - P_discharge[t], name=f"balance_w_battery_{t}")
        m.addConstr(PG[t] + PV_curtail[t] == pv_max[t] * PVcap, name=f"pv_cap_{t}")
        m.addConstr(U[t] >= PD_flex[t] - ref_profile[t], name=f"dev_pos_{t}")
        m.addConstr(U[t] >= -(PD_flex[t] - ref_profile[t]), name=f"dev_neg_{t}")
        
        
    # State of charge constraints
    for t in range(0, T-1):
        m.addConstr(SOC[t+1] == SOC[t] + (P_charge[t] * efficiency_charge) - (P_discharge[t] / efficiency_discharge), name=f"battery_content{t}")


    # Only for 1a   
    #m.addConstr(gp.quicksum(PD[t] for t in range(T)) >= min_daily_energy, name="daily_min_energy")


    # Full objective function
    obj = gp.quicksum(
        energy_price[t] * (PIMP[t] - PEXP[t]) +
        import_tariff * PIMP[t] +
        export_tariff * PEXP[t] + U[t] for t in range(T)
    )
    
    # Set objective 
    m.setObjective(obj, GRB.MINIMIZE)

    # Optimize model
    m.optimize()
    
    # Check if optimization is successful
    if m.Status != GRB.OPTIMAL:
        raise RuntimeError(f"Solver ended with status {m.Status}")


    # Extract results
    results = []
    for t in range(T):
        results.append([
            t,
            PIMP[t].X,
            PEXP[t].X,
            PG[t].X,
            PD_flex[t].X,
            U[t].X,
            PIMP[t].X - PEXP[t].X,
            PV_curtail[t].X,
            (SOC[t].X/battery_capacity)*100,
            
        ])
    return results, m.ObjVal


In [56]:
#def main():
# File paths
base_dir = Path().resolve().parent / "data" / "question_1c"
# Load input data
appliance_params1c = load_json(base_dir / "appliance_params.json")
bus_params1c = load_json(base_dir / "bus_params.json")
der_production1c = load_json(base_dir / "DER_production.json")
usage_preference1c = load_json(base_dir / "usage_preferences.json")

# Run optimization
results_battery1c, obj_val_battery1c = run_flex_load_pv_optimization_w_battery(bus_params1c, der_production1c, usage_preference1c, appliance_params1c)
print(f"\n1.b Objective (cost + discomfort): {obj_val_battery1c:.3f} DKK\n")
print("Hour | Import | Export | PV_used | Demand | Deviation | NetGrid | PV_Curtail | SOC (%)")
print("-" * 90)
for row in results_battery1c:
    print(f"{row[0]:>4} | {row[1]:>6.3f} | {row[2]:>6.3f} | {row[3]:>7.3f} | {row[4]:>6.3f} | {row[5]:>9.3f} | {row[6]:>7.3f} | {row[7]:>9.3f} | {row[8]:>9.3f}")




if __name__ == "__main__":
    main()


Gurobi Optimizer version 12.0.1 build v12.0.1rc0 (win64 - Windows 11.0 (26100.2))

CPU model: AMD Ryzen 5 6600U with Radeon Graphics, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 121 rows, 240 columns and 383 nonzeros
Model fingerprint: 0x4f698030
Coefficient statistics:
  Matrix range     [9e-01, 1e+00]
  Objective range  [4e-01, 3e+00]
  Bounds range     [2e-01, 1e+03]
  RHS range        [1e-01, 3e+00]
Presolve removed 77 rows and 132 columns
Presolve time: 0.01s
Presolved: 44 rows, 108 columns, 174 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0   -9.4404604e+03   9.028219e+03   0.000000e+00      0s
      65   -1.6728840e+01   0.000000e+00   0.000000e+00      0s

Solved in 65 iterations and 0.01 seconds (0.00 work units)
Optimal objective -1.672884000e+01

1.b Objective (cost + discomfort): -16.729 DKK

Hour | Import | Export | PV_used | Demand | Deviation | Net